<a href="https://colab.research.google.com/github/ingvrkm/deep_learning/blob/main/final_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ก่อนเริ่มการทดลอง ตรวจสอบเวอร์ชันของ PyTorch และ Transformers เพื่อให้แน่ใจว่าสภาพแวดล้อมพร้อมใช้งาน
#import torch
#import transformers

#print(torch.__version__)
#print(transformers.__version__)

# ขั้นตอนที่ 1: ติดตั้ง Library ที่จำเป็นสำหรับการทำงานของโมเดล Deep Learning

ในขั้นตอนแรกนี้ เราจะติดตั้งไลบรารีหลักที่ใช้ในการพัฒนาโมเดลจำแนกอารมณ์ของข้อความรีวิวภาพยนตร์ (Sentiment Analysis) โดยใช้ Deep Learning

- PyTorch: ใช้สำหรับสร้างโมเดล Neural Network และฝึกสอนโมเดล

- Torchvision / Torchaudio: ใช้เสริมสำหรับจัดการข้อมูลภาพหรือเสียง (แม้ในงานนี้จะไม่ใช้โดยตรง แต่จำเป็นต่อการติดตั้งบางส่วนของ PyTorch)

- Transformers: ใช้สำหรับเรียกใช้งาน tokenizer และ pre-trained embeddings จาก HuggingFace

- Datasets: ใช้โหลดและจัดการข้อมูลรีวิวจากแหล่งข้อมูลมาตรฐาน เช่น IMDB

การติดตั้งนี้ทำให้เรามีสภาพแวดล้อมพร้อมสำหรับการสร้างและฝึกโมเดลในขั้นตอนถัดไป

In [2]:
# ติดตั้ง PyTorch และโมดูลเสริมสำหรับงาน Deep Learning
!pip install torch torchvision torchaudio

# ติดตั้ง HuggingFace Transformers สำหรับใช้ tokenizer และโมเดลภาษาที่ผ่านการฝึกมาแล้ว
!pip install transformers

# ติดตั้ง HuggingFace Datasets สำหรับโหลดข้อมูล IMDB Reviews
!pip install datasets

# ขั้นตอนที่ 2: นำเข้าไลบรารีที่จำเป็น (Import Libraries)

หลังจากติดตั้งแพ็กเกจทั้งหมดเรียบร้อยแล้ว ขั้นตอนนี้คือการนำเข้า (import) ไลบรารีหลักที่จะใช้ตลอดทั้งโปรเจค โดยแต่ละตัวมีหน้าที่ดังนี้:

- torch, torch.nn, torch.optim, torch.utils.data → สำหรับสร้างและจัดการโมเดล Neural Network ด้วย PyTorch

- torch.nn.functional (F) → ใช้งานฟังก์ชันเช่น activation functions (ReLU, sigmoid, softmax)

- transformers → ใช้สำหรับโหลด tokenizer และ pre-trained models (เช่น BERT)

- datasets → สำหรับโหลดชุดข้อมูล IMDB Review

- numpy, random, re → ใช้จัดการข้อมูล, ควบคุมค่า seed เพื่อให้ได้ผลลัพธ์ซ้ำได้, และจัดการข้อความด้วย Regular Expression

การ import ไลบรารีทั้งหมดนี้เป็นการเตรียมเครื่องมือพื้นฐานสำหรับการสร้างโมเดล deep learning ของเราในขั้นตอนต่อไป

In [3]:
# PyTorch Core
import torch                                      # ไลบรารีหลักของ PyTorch สำหรับ tensor และ deep learning
import torch.nn as nn                             # ใช้สร้างเลเยอร์และโมเดล Neural Network
import torch.optim as optim                       # ใช้ในการเลือก optimizer เช่น Adam, SGD
from torch.utils.data import Dataset, DataLoader  # ใช้สร้างชุดข้อมูลแบบ custom และโหลดเป็น batch
import torch.nn.functional as F                   # ใช้เรียก activation function เช่น ReLU, Sigmoid, Softmax

# HuggingFace Transformers & Datasets
from transformers import AutoTokenizer, AutoModel # ใช้โหลด tokenizer และโมเดลภาษาที่ผ่านการ pretrain แล้ว
from datasets import load_dataset                 # ใช้โหลดชุดข้อมูลสำเร็จรูป เช่น IMDB Reviews

# Utilities
import numpy as np                                # สำหรับการคำนวณเชิงตัวเลข เช่น การแปลง array
import random                                     # สำหรับสุ่มข้อมูล และกำหนดค่า seed
import re                                         # สำหรับจัดการข้อความด้วย Regular Expression (เช่น การลบอักขระพิเศษ)

# ขั้นตอนที่ 3: โหลดและสำรวจข้อมูล (Dataset Loading & Exploration)

ในขั้นตอนนี้ เราจะใช้ไลบรารี HuggingFace Datasets เพื่อโหลดชุดข้อมูลมาตรฐานที่ชื่อว่า IMDB Dataset ซึ่งเป็นชุดข้อมูลรีวิวภาพยนตร์ที่ถูกติดป้ายกำกับอารมณ์ (sentiment) ไว้แล้วว่า

- 0 = รีวิวเชิงลบ (negative)

- 1 = รีวิวเชิงบวก (positive)

ชุดข้อมูลนี้มีตัวอย่างข้อความขนาดใหญ่ เหมาะกับการฝึกสอนโมเดล deep learning สำหรับงาน Sentiment Analysis โดยเฉพาะ
หลังจากโหลดข้อมูลแล้ว เราจะลองพิมพ์ตัวอย่างข้อมูลบางส่วนจากชุด train เพื่อดูโครงสร้างจริงของ dataset ก่อนเข้าสู่ขั้นตอน preprocessing

In [4]:
# โหลดชุดข้อมูล IMDB ผ่าน HuggingFace datasets
# ซึ่งมีข้อมูลรีวิวภาพยนตร์พร้อมป้ายกำกับ sentiment (positive / negative)
dataset = load_dataset("imdb")

# แสดงตัวอย่างข้อมูลจากชุด train เพื่อตรวจสอบโครงสร้างของข้อมูล
# dataset['train'][0] จะคืนค่าเป็น dictionary เช่น:
# {'text': 'This movie was amazing...', 'label': 1}
print(dataset['train'][0])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

# ขั้นตอนที่ 4: การเตรียมข้อมูลข้อความ (Text Preprocessing)

ข้อความรีวิวในชุดข้อมูล IMDB มักมีอักขระพิเศษ สัญลักษณ์ หรือเครื่องหมายวรรคตอนที่ไม่จำเป็นต่อการเรียนรู้ของโมเดล Deep Learning
ดังนั้นก่อนที่จะนำข้อมูลเข้า Tokenizer เราจำเป็นต้องทำการ Preprocessing เพื่อทำให้ข้อความอยู่ในรูปแบบที่สะอาดและเหมาะสมมากขึ้น โดยในขั้นตอนนี้เราจะทำการ:

- แปลงข้อความให้เป็นตัวพิมพ์เล็กทั้งหมด (lowercase) เพื่อให้คำเหมือนกันมีรูปแบบเดียวกัน

- ลบอักขระพิเศษและสัญลักษณ์ที่ไม่จำเป็น เช่น !, ., ?, , เป็นต้น

จากนั้นใช้คำสั่ง dataset.map() เพื่อให้ฟังก์ชัน clean_text() ทำงานกับทุกตัวอย่างในชุดข้อมูลอย่างมีประสิทธิภาพ
ผลลัพธ์คือชุดข้อมูลที่ข้อความได้รับการทำความสะอาดเรียบร้อย พร้อมเข้าสู่กระบวนการ Tokenization ในขั้นตอนถัดไป

In [5]:
# สร้างฟังก์ชันสำหรับทำความสะอาดข้อความ
def clean_text(text):
    text = text.lower()                         # แปลงตัวอักษรทั้งหมดให้เป็นตัวพิมพ์เล็ก
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)  # ลบอักขระพิเศษและสัญลักษณ์ที่ไม่จำเป็นออก
    return text

# ใช้คำสั่ง .map() เพื่อให้ฟังก์ชัน clean_text() ทำงานกับทุกตัวอย่างใน dataset
# จะสร้างคอลัมน์ 'text' ใหม่ที่ผ่านการทำความสะอาดแล้ว
dataset = dataset.map(lambda x: {'text': clean_text(x['text'])})

# ตรวจสอบตัวอย่างหลังจากทำความสะอาดข้อความ
print(dataset['train'][0])

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

# ขั้นตอนที่ 5: Tokenization และการเตรียมข้อมูลสำหรับโมเดล (Data Preparation)

หลังจากที่เราทำความสะอาดข้อความเรียบร้อยแล้ว ขั้นตอนต่อมาคือการ แปลงข้อความให้กลายเป็นตัวเลข (Tokenization) เพื่อให้โมเดล Deep Learning สามารถเรียนรู้ได้

ในที่นี้เราใช้ Tokenizer จาก HuggingFace (BERT-base-uncased) ซึ่งทำหน้าที่

- แปลงคำในข้อความให้เป็น token IDs ตามพจนานุกรมของ BERT

- กำหนดความยาวสูงสุด (max_length=256)

- เติม padding เพื่อให้ความยาวของทุกข้อความเท่ากัน

- ตัดข้อความที่ยาวเกินไป (truncation)

จากนั้นเราจะนำผลลัพธ์ที่ได้มาสร้างเป็น Custom PyTorch Dataset เพื่อให้ DataLoader สามารถสุ่มชุดข้อมูลทีละ batch สำหรับใช้ฝึกสอนโมเดลได้สะดวก
ขั้นตอนนี้ถือเป็นการเตรียมข้อมูลเข้าสู่กระบวนการ Deep Learning อย่างสมบูรณ์

In [7]:
# โหลด Tokenizer จากโมเดล BERT-base-uncased (HuggingFace)
# ทำหน้าที่แปลงข้อความเป็น token id เพื่อให้โมเดลสามารถอ่านข้อมูลได้
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# ฟังก์ชันสำหรับ encode ข้อความแต่ละชุดให้เป็นตัวเลข
def encode(examples):
    return tokenizer(examples['text'],
                     padding='max_length',  # เติม padding ให้ความยาวเท่ากัน
                     truncation=True,       # ตัดข้อความที่ยาวเกินขนาดที่กำหนด
                     max_length=256         # จำกัดความยาวสูงสุดของข้อความต่อหนึ่งตัวอย่าง
           )


# ใช้ tokenizer กับทุกตัวอย่างใน dataset
dataset = dataset.map(encode, batched=True)

# สร้าง Custom PyTorch Dataset Class
class IMDBDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # แปลงข้อมูลแต่ละ batch เป็น tensor เพื่อใช้กับ PyTorch model
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

# แยก dataset ออกเป็น train และ test
train_dataset = IMDBDataset(dataset['train'], dataset['train']['label'])
test_dataset = IMDBDataset(dataset['test'], dataset['test']['label'])

# ใช้ DataLoader สำหรับการสุ่มข้อมูล batch ละ 16 ตัวอย่าง
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

# ขั้นตอนที่ 6: ปรับปรุงโครงสร้าง Dataset สำหรับ PyTorch (Custom Dataset Class)

หลังจากที่เราทำการ Tokenize ชุดข้อมูลเรียบร้อยแล้ว ผลลัพธ์ที่ได้จาก HuggingFace จะอยู่ในรูปแบบของ dictionary ที่มี key เช่น input_ids, attention_mask, และ label ซึ่งยังไม่สามารถนำไปใช้กับ PyTorch ได้โดยตรง

ในขั้นตอนนี้ เราจะสร้างคลาส IMDBDataset ขึ้นมาใหม่ เพื่อแปลงข้อมูลเหล่านี้ให้กลายเป็น PyTorch Tensor ที่สามารถโหลดเข้าสู่โมเดลผ่าน DataLoader ได้อย่างมีประสิทธิภาพ

การออกแบบคลาส Dataset เองทำให้เราสามารถควบคุมรูปแบบของข้อมูลที่ส่งเข้าโมเดลได้ และเป็นพื้นฐานสำคัญของการเทรนโมเดล deep learning ด้วย PyTorch

In [8]:
# สร้างคลาส IMDBDataset เพื่อแปลงข้อมูลจาก HuggingFace dataset
# ให้อยู่ในรูปแบบที่ PyTorch สามารถอ่านได้โดยตรง
class IMDBDataset(Dataset):
    def __init__(self, hf_dataset):
        # แปลง field ต่าง ๆ เป็น tensor
        self.input_ids = torch.tensor(hf_dataset['input_ids'])
        self.attention_mask = torch.tensor(hf_dataset['attention_mask'])
        self.labels = torch.tensor(hf_dataset['label'])

    def __len__(self):
        # คืนค่าจำนวนตัวอย่างทั้งหมดใน dataset
        return len(self.labels)

    def __getitem__(self, idx):
        # คืนค่าข้อมูลแต่ละตัวอย่างในรูปแบบ dictionary
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx]
        }

# สร้างชุดข้อมูล train และ test จาก dataset ที่ encode แล้ว
train_dataset = IMDBDataset(dataset['train'])
test_dataset = IMDBDataset(dataset['test'])

# สร้าง DataLoader สำหรับการเทรนและทดสอบ
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


# ขั้นตอนที่ 7: ออกแบบโมเดล Deep Learning (LSTM Classifier)

ในขั้นตอนนี้ เราจะสร้างโมเดลแบบ Recurrent Neural Network (RNN) โดยใช้ Long Short-Term Memory (LSTM) ซึ่งเป็นสถาปัตยกรรมที่เหมาะสมสำหรับการประมวลผลข้อมูลลำดับ (sequence) เช่น ข้อความ (text)

โมเดลที่ออกแบบจะมีโครงสร้างหลัก 3 ส่วน:

1. Embedding Layer — แปลง token id เป็นเวกเตอร์เชิงความหมาย

2. LSTM Layer — เรียนรู้ความสัมพันธ์ของคำตามลำดับเวลา

3. Fully Connected Layer — ใช้ผลลัพธ์สุดท้ายของ LSTM ในการจำแนกอารมณ์ (positive/negative)

การเลือกใช้ LSTM ทำให้โมเดลสามารถจดจำบริบทของข้อความได้ดีกว่า RNN ทั่วไป และเป็นทางเลือกที่เหมาะสมสำหรับงาน Sentiment Analysis

In [9]:
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim, n_layers=1):
        super(LSTMClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)                     # Embedding Layer: แปลง token ID เป็นเวกเตอร์เชิงความหมาย
        self.lstm = nn.LSTM(embed_dim, hidden_dim, n_layers, batch_first=True)   # LSTM Layer: ประมวลผลลำดับข้อความด้วย hidden_dim และ n_layers ที่กำหนด
        self.fc = nn.Linear(hidden_dim, output_dim)                              # Fully Connected Layer: แปลง hidden state สุดท้ายเป็นผลลัพธ์การจำแนก (positive/negative)

    def forward(self, x):
        x = self.embedding(x)        # แปลง token ID เป็น embedding vector
        _, (hn, _) = self.lstm(x)    # ดึง hidden state สุดท้ายจาก LSTM
        hn = hn[-1]                  # ใช้ hidden state จาก layer สุดท้ายเพื่อเป็นตัวแทนของประโยค
        out = self.fc(hn)            # ส่งผ่าน fully connected layer เพื่อจำแนกอารมณ์
        return out

  # โมเดลนี้ประกอบด้วย Embedding → LSTM → Fully Connected เหมาะสำหรับงาน Sentiment Analysis ที่เน้นลำดับของคำ

# ขั้นตอนที่ 8: ตั้งค่าอุปกรณ์ สร้างโมเดล และฝึกสอน (Training Loop)

ในขั้นตอนนี้ เราจะเริ่ม “สอน” โมเดลที่ออกแบบไว้ (LSTMClassifier) ให้เรียนรู้จากข้อมูลรีวิวภาพยนตร์
โดยจะทำการ

- ตรวจสอบอุปกรณ์ที่ใช้ (GPU / CPU) เพื่อเร่งการคำนวณ

- กำหนดค่าพารามิเตอร์หลัก เช่น vocab_size, embedding_dim, hidden_dim และ output_dim

- ตั้งค่า optimizer (ใช้ Adam) และ loss function (ใช้ CrossEntropyLoss สำหรับการจำแนก 2 คลาส)

- รัน training loop เพื่ออัปเดตน้ำหนักของโมเดลในแต่ละ epoch

- แสดงค่า loss ของแต่ละรอบเพื่อประเมินความสามารถของโมเดลในการเรียนรู้

โมเดลจะค่อย ๆ ปรับค่าพารามิเตอร์ภายในผ่านกระบวนการ backpropagation
จนสามารถจำแนกรีวิวได้แม่นยำยิ่งขึ้นในแต่ละ epoch

In [10]:
# ใช้ GPU หากมีให้ใช้งาน เพื่อเพิ่มความเร็วในการเทรน
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# กำหนดขนาดของคำศัพท์ (vocab size) จาก tokenizer
vocab_size = tokenizer.vocab_size if hasattr(tokenizer, 'vocab_size') else tokenizer.get_vocab_size()

# สร้างโมเดล LSTMClassifier โดยกำหนดขนาด embedding, hidden layer และ output
model = LSTMClassifier(
    vocab_size=vocab_size,
    embed_dim=128,
    hidden_dim=128,
    output_dim=2
)

# ย้ายโมเดลไปที่ GPU หรือ CPU ตามที่ตรวจพบ
model.to(device)

# กำหนด optimizer และ loss function
optimizer = optim.Adam(model.parameters(), lr=1e-3)  # ใช้ Adam optimizer สำหรับปรับค่า weight
criterion = nn.CrossEntropyLoss()                    # ใช้ CrossEntropy สำหรับการจำแนก 2 class (positive / negative)

# Training loop
for epoch in range(3):                               # เทรนทั้งหมด 3 รอบ (epochs)
    model.train()                                    # ตั้งค่าโมเดลให้อยู่ในโหมด train
    total_loss = 0
    for batch in train_loader:                       # ดึงข้อมูลแบบ batch
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()                        # เคลียร์ gradient เดิมก่อนคำนวณรอบใหม่
        outputs = model(input_ids)                   # ส่งข้อมูลเข้าโมเดลเพื่อให้โมเดลทำนาย
        loss = criterion(outputs, labels)            # คำนวณค่าความสูญเสีย (loss)
        loss.backward()                              # คำนวณ gradient ย้อนกลับ
        optimizer.step()                             # ปรับค่าพารามิเตอร์ของโมเดลตาม gradient

        total_loss += loss.item()                    # สะสมค่า loss ของแต่ละ batch

    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")  # แสดงผลค่า loss เฉลี่ยต่อ epoch

Epoch 1, Loss: 0.6940
Epoch 2, Loss: 0.6890
Epoch 3, Loss: 0.5615


# ขั้นตอนที่ 9: ประเมินผลโมเดล (Model Evaluation)

หลังจากเทรนโมเดลเสร็จแล้ว ขั้นตอนนี้จะทำการ ทดสอบโมเดลกับชุดข้อมูลที่ไม่เคยเห็นมาก่อน (test set)
เพื่อประเมินความสามารถของโมเดลในการจำแนกอารมณ์ของข้อความใหม่ ๆ

ขั้นตอนนี้จะ

- ตั้งค่าโมเดลให้อยู่ในโหมด eval() เพื่อปิดการอัปเดตค่า weight

- เก็บผลการทำนาย (preds) และคำตอบจริง (labels)

- คำนวณค่า Accuracy เพื่อดูว่าผลทำนายถูกต้องกี่เปอร์เซ็นต์

- แสดงผลลัพธ์สุดท้ายเป็นค่าความแม่นยำของโมเดล

In [11]:
# ใช้คำนวณค่า Accuracy ของโมเดล
from sklearn.metrics import accuracy_score

model.eval()       # ตั้งค่าโมเดลให้อยู่ในโหมดประเมินผล (ไม่อัปเดต weight)
all_preds = []     # สร้าง list สำหรับเก็บผลทำนาย
all_labels = []    # สร้าง list สำหรับเก็บคำตอบจริง

with torch.no_grad():                                # ปิด gradient เพื่อประหยัดหน่วยความจำและคำนวณเร็วขึ้น
    for batch in test_loader:                        # วนลูปผ่าน test set ทีละ batch
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids)                   # ส่งข้อมูลเข้าโมเดลเพื่อให้ทำนาย
        preds = torch.argmax(outputs, dim=1)         # เลือกคลาสที่มีค่าความน่าจะเป็นสูงสุด
        all_preds.extend(preds.cpu().numpy())        # เก็บผลทำนาย
        all_labels.extend(labels.cpu().numpy())      # เก็บคำตอบจริง

accuracy = accuracy_score(all_labels, all_preds)     # คำนวณ Accuracy ทั้งหมด
print("Test Accuracy:", accuracy)

Test Accuracy: 0.80804


# ขั้นตอนที่ 10: ทดสอบการทำงานของโมเดลกับข้อความใหม่ (Model Inference)

ในขั้นตอนสุดท้ายนี้ เราจะนำโมเดลที่ผ่านการฝึกสอนแล้วมาทดลองใช้งานจริง
โดยเขียนฟังก์ชัน predict_sentiment() เพื่อให้ผู้ใช้สามารถพิมพ์รีวิวภาพยนตร์ได้เอง
จากนั้นโมเดลจะวิเคราะห์และทำนายว่ารีวิวนั้นมี อารมณ์บวก (Positive) หรือ อารมณ์ลบ (Negative)

ขั้นตอนนี้จึงเป็นเหมือน “การทดสอบภาคสนาม” ของโมเดล
เพื่อดูว่าโมเดลสามารถทำงานได้ดีในสถานการณ์จริงนอกเหนือจากชุดข้อมูลฝึกหรือไม่

In [12]:
def predict_sentiment(text):
    model.eval()                                          # ตั้งค่าโมเดลให้อยู่ในโหมดประเมินผล
    tokens = tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=256) # แปลงข้อความเป็น token ตามรูปแบบที่โมเดลใช้
    tokens = {k:v.to(device) for k,v in tokens.items()}   # ย้ายข้อมูลไปที่อุปกรณ์เดียวกับโมเดล (CPU/GPU)
    with torch.no_grad():                                 # ปิด gradient เพื่อให้คำนวณเร็วขึ้น
        output = model(tokens['input_ids'])               # ส่งข้อความเข้าโมเดลเพื่อให้ทำนาย
        pred = torch.argmax(output, dim=1).item()         # เลือกคลาสที่โมเดลให้ความมั่นใจสูงสุด
    return "Positive" if pred == 1 else "Negative"        # แปลงผลลัพธ์เป็นข้อความที่เข้าใจง่าย

# ทดสอบโมเดลกับรีวิวใหม่
print(predict_sentiment("I really loved this movie!"))    # ควรได้ผลเป็น 'Positive'

Positive


# สรุปผลการทดลอง

โปรเจคนี้ได้พัฒนาโมเดล LSTM สำหรับจำแนกอารมณ์ของข้อความรีวิวภาพยนตร์ โดยใช้ PyTorch และ Tokenizer ในการเตรียมข้อมูล
ผลการทดลองพบว่าโมเดลสามารถแยกรีวิวเชิงบวกและเชิงลบได้อย่างมีประสิทธิภาพ
พร้อมฟังก์ชัน predict_sentiment() ที่สามารถทำนายข้อความใหม่ได้ทันที

